In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import baikalfunctions as bfunc
import scheme_mar2023 as scheme
import mysecure

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
pd.__version__

In [ ]:
# pd.set_option('display.min_rows', 100)
%matplotlib inline
plt.style.use('dark_background')   # try another styles: 'classic'
plt.rcParams['figure.figsize'] = [15, 5]
#plt.subplots_adjust(top=1, left=0, right=1, bottom=0)

In [ ]:
pwd = 'G:\\1_Data1\\89_BaikalFeb2022\\'
saveImgPath = 'G:\\1_Data1\\89_BaikalFeb2022\\'
saveDataPath = 'G:\\1_Data1\\89_BaikalFeb2022\\'

In [ ]:
names=['Date', 'Time', 'DateUTC', 'TimeUTC', 'latitude', 'longitude', 'speed', 'course', 'solar_altitude', 'solar-azimuth',
       'temperature_air', 'pressure_air', 'relative-humidity_air', 'precipitation', 'solar_eye', 'solar_uv',
       'co2_picarro', 'co2_picarro_max', 'co2_picarro_min', 'ch4_picarro', 'ch4_picarro_max', 'ch4_picarro_min', 'h2o_picarro', 'co2_licor', 'h2o_licor', 'o2_water', 'o2_water_temperature',
       'ph', 'temperature_water', 'temperature_equ_1', 'temperature_equ_2', 'temperature_equ_3', 'temperature_equ_4', 'flow_water', 'flow_air', 'channel', 'flag',
       'fluo_nx_red', 'fluo_nx_green', 'fluo_nx_blue', 'fluo_kfa_red', 'fluo_kfa_green', 'fluo_kfa_blue', # 'comment',
       'water_flow_1', 'water_flow_2', 'water_flow_3', 'water_flow_4',
       'temperature_ds_1', 'temperature_ds_2', 'temperature_ds_3', 'temperature_ds_4', 'temperature_ds_5', 'temperature_ds_6', 'temperature_ds_7'
      ]

In [ ]:
df = pd.read_csv(pwd+'origin\\data.txt',
                 sep='\s+',
                 header=None,
                 names=names,
                 index_col=False,
                 parse_dates={'DateTime':['Date', 'Time']},
                 dayfirst=True,
                 na_values='-999999',
                 decimal=',',
                 nrows=1440*3*10)
df.info()

In [ ]:
df.describe()

### refactor table. remove unused columns, rename columns

In [ ]:
df.drop([f'temperature_equ_{i}' for i in range(10)], axis='columns', inplace=True, errors='ignore')

In [ ]:
df.rename(columns=scheme.NAME_CONV_BAS, inplace=True)
df.sort_values(by='DateTime', inplace=True)
df.reset_index(inplace=True, drop=True)
df.describe()

In [ ]:
df.sort_values(by='DateTime', inplace=True)

In [ ]:
### remove service variables
df.drop([x for x in df.columns if x.lower().endswith('err')], axis='columns', inplace=True, errors='ignore')
df.drop([x for x in df.columns if x.lower().endswith('max')], axis='columns', inplace=True, errors='ignore')
df.drop([x for x in df.columns if x.lower().endswith('min')], axis='columns', inplace=True, errors='ignore')
df.drop([x for x in df.columns if x.lower().startswith('pump')], axis='columns', inplace=True, errors='ignore')
df.drop(['HumidityAir', ], axis='columns', inplace=True, errors='ignore')


In [ ]:
df.shape, df.columns

In [ ]:
plt.rcParams['figure.figsize'] = [15, 15]
fig, axs = plt.subplots(7, 1)

axs[0].set_title('vCO2, ppm')
axs[0].plot(df.DateTime, df['vCO2'], '-', c='silver')

axs[1].set_title('vCH4, ppm')
axs[1].plot(df.DateTime, df['vCH4'], '-', c='silver')

axs[2].set_title('Temp')
axs[2].plot(df.DateTime, df['TempEqu1'], 'r-')
axs[2].plot(df.DateTime, df['TempEqu2'], 'g-')
axs[2].plot(df.DateTime, df['TempEqu3'], 'b-')
axs[2].plot(df.DateTime, df['TempAir'], 'c-')
axs[2].set_ylim(-2, 10)

axs[3].set_title('WaterFlow')
axs[3].plot(df.DateTime, df['WaterFlowEqu1'], 'r-')
axs[3].plot(df.DateTime, df['WaterFlowEqu2'], 'g-')
axs[3].plot(df.DateTime, df['WaterFlowEqu3'], 'b-')

axs[4].set_title('Press')
axs[4].plot(df.DateTime, df['PressAir'], 'c-')
axs[5].set_title('Solar')
axs[5].plot(df.DateTime, df['LightLX'], 'w-')
axs[6].set_title('SolarUV')
axs[6].plot(df.DateTime, df['LightUV'], 'b-')



### Correction data  
 ** Be carefull !!!  Make it ONCE !!! **


In [ ]:
# gether/apply specific variables
### data corrections
# df['vCO2'] = df['vCO2'] * scheme.CO2PicarroScaut_cor[0] + scheme.CO2PicarroScaut_cor[1]
# df['vCH4'] = df['vCH4'] * scheme.CH4PicarroScaut_cor[0] + scheme.CH4PicarroScaut_cor[1]
# df['vCO2'] = df['CO2_dry']
# df['vCH4'] = df['CH4_dry']
df['LightLX'] = df['LightLX'] * scheme.solar_cor[0] + scheme.solar_cor[1]
df['LightUV'] = df['LightUV'] * scheme.solar_uv_cor[0] + scheme.solar_uv_cor[1]
df['PressAir'] = bfunc.pressConvert(df['PressAir'])
df['PressAir'] = df['PressAir'] * scheme.press_cor[0] + scheme.press_cor[1]

### servise data corrections
df['TempEqu1'] = df['TempEqu1'] * scheme.tempEqu1_cor[0] + scheme.tempEqu1_cor[1]
df['TempEqu2'] = df['TempEqu2'] * scheme.tempEqu2_cor[0] + scheme.tempEqu2_cor[1]
df['TempEqu3'] = df['TempEqu3'] * scheme.tempEqu3_cor[0] + scheme.tempEqu3_cor[1]

df['AirFlow'] = df['AirFlow'] * scheme.airflow_cor[0] + scheme.airflow_cor[1]
df['WaterFlowEqu1'] = df['WaterFlowEqu1'] * scheme.waterflowEqu1_cor[0] + scheme.waterflowEqu1_cor[1]
df['WaterFlowEqu2'] = df['WaterFlowEqu2'] * scheme.waterflowEqu2_cor[0] + scheme.waterflowEqu2_cor[1]
df['WaterFlowEqu3'] = df['WaterFlowEqu3'] * scheme.waterflowEqu3_cor[0] + scheme.waterflowEqu3_cor[1]


###  Remove bad data   
List of accidental cases to filter data (see notes)

In [ ]:
filter_list = [{'date_start': '19.02.2022 13:00', 'date_stop': '19.02.2022 13:32', 'cols': ['vCO2', 'vCH4'], 'fill_with': np.NaN},   # Picarro is not ready
               {'date_start': '20.02.2022 17:53', 'date_stop': '20.02.2022 17:55', 'cols': ['vCO2', 'vCH4'], 'fill_with': np.NaN},   # Picarro mulfunction
               {'date_start': '21.02.2022 07:14', 'date_stop': '21.02.2022 11:52', 'cols': ['vCO2', 'vCH4'], 'fill_with': np.NaN},   # Picarro mulfunction
               {'date_start': '20.02.2022 18:14', 'date_stop': '21.02.2022 02:30', 'cols': ['vCH4'], 'fill_with': np.NaN},   # Picarro mulfunction
              ]

for cycle in filter_list:
    for col in cycle['cols']:
        date_start = pd.to_datetime(cycle['date_start'], dayfirst=True)
        date_stop = pd.to_datetime(cycle['date_stop'], dayfirst=True)
        df.loc[(df['DateTime'] > date_start) & (df['DateTime'] < date_stop), col] = cycle['fill_with']

In [ ]:
cols = ['FluoNxRed', 'FluoNxGrn', 'FluoNxBlu', 'FluoKfaRed', 'FluoKfaGrn', 'FluoKfaBlu']
for col in cols:
    df.loc[df[col] == 0, col] = np.NaN
cols = ['WaterFlowEqu1', 'WaterFlowEqu2', 'WaterFlowEqu3', 'AirFlow']
for col in cols:
    df.loc[df[col] < 0, col] = np.NaN

### prepare data

In [ ]:
v_state_list = list(scheme.CHANNEL_COLS.values())

chnl = 'Channel'
if not chnl in df.columns:
    df[chnl] = 0
if 'V1_state' in df.columns:
    df.loc[df.V1_state == 1, chnl] = 1
    df.loc[df.V2_state == 1, chnl] = 2
    df.loc[df.V3_state == 1, chnl] = 3
    df.loc[df.V4_state == 1, chnl] = 4
    df.loc[df.V5_state == 1, chnl] = 5
    df.loc[df.V6_state == 1, chnl] = 6
df.drop(v_state_list, inplace=True, errors='ignore')

#### split data on channel events
separate `CH4air` from whole massive. take minimum from each cycle

In [ ]:
df['event'] = 0
df.loc[df[chnl] != df[chnl].shift(1), ['event']] = 1
df['event'] = df['event'].cumsum()
df

In [ ]:
dfr = df.groupby(by=['event']).min()
dfr = dfr.loc[dfr[chnl] == 6,['DateTime', 'vCH4']]
dfr

In [ ]:
df = pd.merge(left=df, right=dfr, left_on='DateTime', right_on='DateTime', how='left', suffixes=(None, 'air'))
df

In [ ]:
ch_v = 'Chn_valid'
df[ch_v] = 1
for deep in range(1, 5):
    df.loc[df[chnl] != df[chnl].shift(deep), ch_v] = 0

In [ ]:
df['vCO2air'] = df['vCO2'][(df[chnl] == 6) & (df[ch_v] == 1)]
# df['vCH4air'] = df['vCH4'][(df[chnl] == 6) & (df[ch_v] == 1)]
for col in ['vCO2air',]:
    df[col] = df[col].rolling(3).mean()

df['vCO2air'].interpolate(method='values', inplace=True)
df['vCH4air'].interpolate(method='values', inplace=True)  ## `time` method mb better
df['pCO2air'] = df['vCO2air'] * df['PressAir']  # mkatm
df['pCH4air'] = df['vCH4air'] * df['PressAir']  # mkatm

In [ ]:
df['vCH4air'].describe()

In [ ]:
df

In [ ]:
## do not know why is it here
df = df.resample('20S', on='DateTime').mean()
df[chnl].fillna(method='ffill', inplace=True)

In [ ]:
df['DateTime'] = df.index

In [ ]:
df

In [ ]:
plt.rcParams['figure.figsize'] = [15, 5]
fig, axs = plt.subplots(2, 1)

axs[0].set_title('CO2 air, ppm')
axs[0].set_ylim(400, 500)
axs[0].plot(df.DateTime, df['vCO2'], '-', c='silver')
axs[0].plot(df.DateTime, df['vCO2air'], 'r-', alpha=0.2)

axs[1].set_title('CH4 air, ppm')
axs[1].set_ylim(1.5, 3)
# axs[1].set_xlim(pd.to_datetime('2022-02-25 12:00'), pd.to_datetime('2022-02-25 14:00'))
axs[1].plot(df.DateTime, df['vCH4'], '-', c='silver')
axs[1].plot(df.DateTime, df['vCH4air'], 'b-')
fig.savefig(saveImgPath+'cAir_vs_time.png', transparent=False)

In [ ]:
df.columns

## RECOVERY !!!

In [ ]:
df['DateSec'] = df['DateTime'].astype('int64')//10**9
df['dTSec'] = df['DateSec'] - df['DateSec'].shift(1)
dt = df['dTSec'] / 60   ## delta time, min
ch_v = 'Chn_valid'
df[ch_v] = 1
for deep in range(1, 7):
    df.loc[df[chnl] != df[chnl].shift(deep), ch_v] = 0

#### recovery `CO2/CH4`, channel `1` (bottom in Feb 2022)

In [ ]:
equ_vol = scheme.equ_walltube_param['equ_vol']  # equivalent equ volume, l
equ_cap = scheme.equ_walltube_param['equ_cap']  # equilibrator capacity
wtr_flow_min = scheme.equ_walltube_param['water_flow_min'] 

In [ ]:
t_wtr = df['TempEqu1']
t_air = t_wtr
wtr_flow = df['WaterFlowEqu1']
air_flow = df['AirFlow']

In [ ]:
t_wtr

In [ ]:
### CO2 water
df['vCO2equ'] = df['vCO2'][(df[chnl] == 1) & (df[ch_v] == 1) & (wtr_flow > wtr_flow_min)]
solubility = bfunc.getSolubility(t_wtr, 'CO2')

density = bfunc.getDensity(t_air, df['PressAir'], 'CO2')
cGasAir = df['pCO2air'] / 1000000 * density
pGasEquAir = df['vCO2equ'] * df['PressAir']
cGasEquAir = pGasEquAir * density / 1000000  # g/l

tau = equ_vol/(air_flow+wtr_flow*equ_cap*solubility/density)
eternal = (cGasEquAir-cGasEquAir.shift(1)*np.exp(-1*dt/tau))/(1-np.exp(-1*dt/tau))
cGasWtr = (eternal*(wtr_flow*equ_cap*solubility/density+air_flow)-air_flow*cGasAir)/(wtr_flow*equ_cap)

df['cCO2bot'] = cGasWtr * 1000     # mg/l
df['pCO2bot'] = cGasWtr * 1000000 / solubility  # mkatm

In [ ]:
### CH4 water
df['vCH4equ'] = df['vCH4'][(df[chnl] == 1) & (df[ch_v] == 1)]
solubility = bfunc.getSolubility(t_wtr, 'CH4')

density = bfunc.getDensity(t_air, df['PressAir'], 'CH4')
cGasAir = df['pCH4air'] / 1000000 * density
pGasEquAir = df['vCH4equ'] * df['PressAir']
cGasEquAir = pGasEquAir * density / 1000000  # g/l

tau = equ_vol/(air_flow+wtr_flow*equ_cap*solubility/density)
eternal = (cGasEquAir-cGasEquAir.shift(1)*np.exp(-1*dt/tau))/(1-np.exp(-1*dt/tau))
cGasWtr = (eternal*(wtr_flow*equ_cap*solubility/density+air_flow)-air_flow*cGasAir)/(wtr_flow*equ_cap)
df['cCH4bot'] = cGasWtr * 1000000000    # ng/l
df['pCH4bot'] = cGasWtr * 1000000 / solubility  # mkatm

In [ ]:
df['pCH4bot'].describe()

#### recovery `CO2/CH4`, channel `3` (surface in Feb 2022)

In [ ]:
t_wtr = df['TempEqu2']
t_air = t_wtr
wtr_flow = df['WaterFlowEqu2']  # waterflow sensor at channel #2 didn't work properly

In [ ]:
t_wtr.describe()

In [ ]:
### CO2 water
df['vCO2equ'] = df['vCO2'][(df[chnl] == 3) & (df[ch_v] == 1) & (wtr_flow > wtr_flow_min)]
solubility = bfunc.getSolubility(t_wtr, 'CO2')

density = bfunc.getDensity(t_air, df['PressAir'], 'CO2')
cGasAir = df['pCO2air'] / 1000000 * density
pGasEquAir = df['vCO2equ'] * df['PressAir']
cGasEquAir = pGasEquAir * density / 1000000  # g/l

tau = equ_vol/(air_flow+wtr_flow*equ_cap*solubility/density)
eternal = (cGasEquAir-cGasEquAir.shift(1)*np.exp(-1*dt/tau))/(1-np.exp(-1*dt/tau))
cGasWtr = (eternal*(wtr_flow*equ_cap*solubility/density+air_flow)-air_flow*cGasAir)/(wtr_flow*equ_cap)

df['cCO2sur'] = cGasWtr * 1000     # mg/l
df['pCO2sur'] = cGasWtr * 1000000 / solubility  # mkatm

In [ ]:
df['vCO2equ'].describe()

In [ ]:
### CH4 water
df['vCH4equ'] = df['vCH4'][(df[chnl] == 3) & (df[ch_v] == 1)]
solubility = bfunc.getSolubility(t_wtr, 'CH4')

density = bfunc.getDensity(t_air, df['PressAir'], 'CH4')
cGasAir = df['pCH4air'] / 1000000 * density
pGasEquAir = df['vCH4equ'] * df['PressAir']
cGasEquAir = pGasEquAir * density / 1000000  # g/l

tau = equ_vol/(air_flow+wtr_flow*equ_cap*solubility/density)
eternal = (cGasEquAir-cGasEquAir.shift(1)*np.exp(-1*dt/tau))/(1-np.exp(-1*dt/tau))
cGasWtr = (eternal*(wtr_flow*equ_cap*solubility/density+air_flow)-air_flow*cGasAir)/(wtr_flow*equ_cap)
df['cCH4sur'] = cGasWtr * 1000000000    # ng/l
df['pCH4sur'] = cGasWtr * 1000000 / solubility  # mkatm

In [ ]:
## rolling average is applied inplace. Be cafelly, run this cell **ONCE**

for col in ['cCO2sur', 'cCO2bot', 'pCO2sur', 'pCO2bot', 'cCH4sur', 'pCH4sur', 'cCH4bot', 'pCH4bot']:
    df.loc[df[col] == np.inf , col] = np.nan 
    df.loc[df[col] == -np.inf , col] = np.nan
    df.loc[df[col] <= 0 , col] = np.nan
    df[col] = df[col].rolling(3, center=True).mean()

In [ ]:
df['pCH4bot'].describe()

In [ ]:
df['pCH4sur'].describe()

In [ ]:
df['pCO2'] = df['vCO2'] * df['PressAir']

plt.rcParams['figure.figsize'] = [18, 5]
fig, ax = plt.subplots()
## ax.set_xlim(pd.to_datetime('05.06.2023 18:00:00', dayfirst=True), pd.to_datetime('05.06.2023 19:00:00', dayfirst=True))
ax.set_title('CO2 water, mkatm')
ax.set_ylim(0, 1200)
ax.plot(df.DateTime, df['pCO2'], '-', c='silver', alpha=0.2)
ax.plot(df.DateTime, df['pCO2sur'], 'g-')
ax.plot(df.DateTime, df['pCO2bot'], 'r-')
ax.plot(df.DateTime, df['pCO2air'], 'c-')
fig.savefig(saveImgPath+'pCO2wtr_vs_time.png')

In [ ]:
count_recent = 30000
df['pCH4'] = df['vCH4'] * df['PressAir']
plt.rcParams['figure.figsize'] = [18, 5]
fig, ax = plt.subplots()
ax.set_title('CH4 water, mkatm')
# ax.set_xlim(pd.to_datetime('06.06.2023 10:00:00', dayfirst=True), pd.to_datetime('06.06.2023 19:00:00', dayfirst=True))
ax.set_ylim(0, 200)
ax.plot(df.tail(count_recent).DateTime, df.tail(count_recent)['pCH4'], '-', c='silver')
ax.plot(df.tail(count_recent).DateTime, df.tail(count_recent)['pCH4bot'], 'r.')
ax.plot(df.tail(count_recent).DateTime, df.tail(count_recent)['pCH4sur'], 'g.')
ax.plot(df.tail(count_recent).DateTime, df.tail(count_recent)['pCH4air'], 'c.')
fig.savefig(saveImgPath+'pCH4wtr_vs_time.png')

In [ ]:
df.columns

In [ ]:
df['hour'] = df['DateTime'].dt.hour # + df['DateTime'].dt.minute / 60
df['hour']

In [ ]:
cols = ['DateTime', 'hour', 'DateSec', 'TempAir', 'PressAir', 'Precipitation', 'LightLX', 'LightUV',
        'vCO2', 'vCH4', 'vH2O', 'AirFlow', 'Channel',
        'FluoNxRed', 'FluoNxGrn', 'FluoNxBlu', 'FluoKfaRed', 'FluoKfaGrn', 'FluoKfaBlu',
        'WaterFlowEqu1', 'WaterFlowEqu2', 'WaterFlowEqu3', 'WaterFlowEqu4',
        'TempEqu1', 'TempEqu2', 'TempEqu3',
        # 'cCO2chm', 'cCH4chm', 'CO2flux', 'CH4flux',
        'vCO2air', 'vCH4air', 'pCO2air', 'pCH4air',
        'cCO2sur', 'pCO2sur', 'cCH4sur', 'pCH4sur',
        'cCO2bot', 'pCO2bot', 'cCH4bot', 'pCH4bot',
       ]
df = df.loc[:, cols]

In [ ]:
df.to_csv(f'{pwd}BaikalFeb22.dat',
          columns=cols,
          sep='\t',
         )

In [ ]:
func_list = ['mean', 'std']
hourly = df.resample('1H', on='DateTime').agg(func_list)
hourly.to_csv(f'{pwd}BaikalFeb22hourly.dat',
              # columns=cols,
              sep='\t',
             )